In [2]:
from datetime import date
import csv

# Постановка задачи: нам предлагается файл с данными о покупках различных паков в игре W.. 
# Необходимо изучить ассоциативные правила, которые мы можем построить на основании предложенных данных.
# Наибольший интерес вызывают акционные паки.

По ссылке, приведенной ниже, можно посомотреть диаграмму Ганта. Она позволяет наглядно показать акционные/неакционные паки.

https://public.tableau.com/profile/roman2610#!/vizhome/Book_207/Dashboard2

Всевозможные паки:

In [3]:
#social_netork='fb'
pcks=[]
with open('trans.csv', 'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter='|', quotechar='|')
    pcks=[]
    spamreader.next()
    for row in spamreader:
        if (row[5] not in pcks):
            pcks.append(row[5])
    print "Всего паков: ",len(pcks)

Всего паков:  214


Априори алгоритм: (параллельно подготовка словаря для tableau)

In [4]:
import sys

from itertools import chain, combinations
from collections import defaultdict
from optparse import OptionParser


def subsets(arr):
    """ Returns non empty subsets of arr"""
    return chain(*[combinations(arr, i + 1) for i, a in enumerate(arr)])


def returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet):
        """calculates the support for items in the itemSet and returns a subset
       of the itemSet each of whose elements satisfies the minimum support"""
        _itemSet = set()
        localSet = defaultdict(int)

        for item in itemSet:
                for transaction in transactionList:
                        if item.issubset(transaction):
                                freqSet[item] += 1
                                localSet[item] += 1

        for item, count in localSet.items():
                support = float(count)/len(transactionList)

                if support >= minSupport:
                        _itemSet.add(item)

        return _itemSet


def joinSet(itemSet, length):
        """Join a set with itself and returns the n-element itemsets"""
        return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])


def getItemSetTransactionList(data_iterator):
    transactionList = list()
    itemSet = set()
    for record in data_iterator:
        transaction = frozenset(record)
        transactionList.append(transaction)
        for item in transaction:
            itemSet.add(frozenset([item]))              # Generate 1-itemSets
    return itemSet, transactionList


def runApriori(data_iter, minSupport, minConfidence):
    """
    run the apriori algorithm. data_iter is a record iterator
    Return both:
     - items (tuple, support)
     - rules ((pretuple, posttuple), confidence)
    """
    itemSet, transactionList = getItemSetTransactionList(data_iter)

    freqSet = defaultdict(int)
    largeSet = dict()
    # Global dictionary which stores (key=n-itemSets,value=support)
    # which satisfy minSupport

    assocRules = dict()
    # Dictionary which stores Association Rules

    oneCSet = returnItemsWithMinSupport(itemSet,
                                        transactionList,
                                        minSupport,
                                        freqSet)

    currentLSet = oneCSet
    k = 2
    while(currentLSet != set([])):
        largeSet[k-1] = currentLSet
        currentLSet = joinSet(currentLSet, k)
        currentCSet = returnItemsWithMinSupport(currentLSet,
                                                transactionList,
                                                minSupport,
                                                freqSet)
        currentLSet = currentCSet
        k = k + 1

    def getSupport(item):
            """local function which Returns the support of an item"""
            return float(freqSet[item])/len(transactionList)

    toRetItems = []
    for key, value in largeSet.items():
        toRetItems.extend([(tuple(item), getSupport(item))
                           for item in value])

    toRetRules = []
    for key, value in largeSet.items()[1:]:
        for item in value:
            _subsets = map(frozenset, [x for x in subsets(item)])
            for element in _subsets:
                remain = item.difference(element)
                if len(remain) > 0:
                    confidence_lift = []
                    item_A=getSupport(element)
                    item_B=getSupport(remain)
                    confidence_lift.append(getSupport(item)/item_A)#conf - 0
                    if confidence_lift[0] >= minConfidence:
                        confidence_lift.append(getSupport(item)/(item_A*item_B))#lift - 1
                        if  item_A>item_B:
                            confidence_lift.append(item_B)#min(minsup) - 2
                        else:
                            confidence_lift.append(item_A)
                        toRetRules.append(((tuple(element), tuple(remain)),
                                           confidence_lift))
    return toRetItems, toRetRules



def dataFromFile(fname):
        """Function which reads from the file and yields a generator"""
        file_iter = open(fname, 'rU')
        for line in file_iter:
                line = line.strip().rstrip(',')                         # Remove trailing comma
                record = frozenset(line.split(','))
                yield record
                
def printResults(items, rules, rs,it, idc, t_d,start_date,end_date, len_ids): 
                                        #принимает: итемы для минсапа, правила с конф, файл для записи результата,
                                        #название текущего пака, номер айдишника для записи в словарь данных по пакам (tableau),
                                        #словарь, дата начала и конца, кол-во транзакций в периоде
    """prints the generated itemsets sorted by support and the confidence rules sorted by confidence"""
    if len(rules)!=0:
        rs.write("SUP------------SUP------------SUP:\n")
        rs.write("For time boundaries on: "+it+"\n") #показывает периоды какого пака исследуются
        for item, support in sorted(items, key=lambda (item, support): support):
            rs.write("item: %s , %.3f \n" % (str(item), support))
            #print "item: %s , %.3f" % (str(item), support)
        rs.write("\nRULES------------RULES------------RULES:\n")
        for rule, confidence in sorted(rules, key=lambda (rule, confidence): confidence):
            pre, post = rule
            idc=idc+1
            t_d[idc]=[]
            st_st="%s ==> %s"%(str(pre), str(post))
            t_d[idc].append(st_st)#rule 
            t_d[idc].append(confidence[0])#conf
            t_d[idc].append(confidence[1])#lift
            t_d[idc].append(confidence[2])#min minsup
            if "fb_" in str(pre)[:7]:
                if "fb_" in str(post)[:7]:
                    t_d[idc].append("fb") #social_network
                else:
                    t_d[idc].append("cross")
            elif "vk_" in str(pre)[:7]:
                if "vk_" in str(post)[:7]:
                    t_d[idc].append("vk") #social_network
                else:
                    t_d[idc].append("cross")
            elif "ok_" in str(pre)[:7]:
                if "ok_" in str(post)[:7]:
                    t_d[idc].append("ok") #social_network
                else:
                    t_d[idc].append("cross")
            elif "mm_" in str(pre)[:7]:
                if "mm_" in str(post)[:7]:
                    t_d[idc].append("mm") #social_network
                else:
                    t_d[idc].append("cross")
            else:
                t_d[idc].append(" ")
            date_for_period=str(date.fromtimestamp(start_date))+" to "+str(date.fromtimestamp(end_date))
            t_d[idc].append(date_for_period)#date
            t_d[idc].append(len_ids)# number of transactions in period
            
            l_s=str(pre)
            r_s=str(post)
            access_act = False
            l=l_s.strip("(").strip(")").strip(",").split(",")
            for i in range(0,len(l)):
                l[i]=l[i].strip("'")
                if l[i] in action_pack:
                    access_act=True
            r=r_s.strip("(").strip(")").strip(",").split(",")
            for i in range(0,len(r)):
                r[i]=r[i].strip("'")
                if r[i] in action_pack:
                    access_act=True
            if access_act==True:
                t_d[idc].append("y")# if rule consist action pack
            else:
                t_d[idc].append("n")
            rs.write("Rule: %s ==> %s , confidence: %.3f, lift: %.3f, min(minsup) %.3f\n" % (str(pre), str(post), confidence[0], confidence[1], confidence[2]))
        rs.write("\n-----------------------------------------\n")
        rs.write("\n")
    return idc, t_d


if __name__ == "__main__":

    optparser = OptionParser()
    optparser.add_option('-f', '--inputFile',
                         dest='input',
                         help='filename containing csv',
                         default=None)
    optparser.add_option('-s', '--minSupport',
                         dest='minS',
                         help='minimum support value',
                         default=0.15,
                         type='float')
    optparser.add_option('-c', '--minConfidence',
                         dest='minC',
                         help='minimum confidence value',
                         default=0.6,
                         type='float')

    (options, args) = optparser.parse_args()


    inFile = dataFromFile("without_bound.csv")

    minSupport = options.minS
    minConfidence = options.minC

    items, rules = runApriori(inFile, minSupport, minConfidence)

    #printResults(items, rules)

Создаем словарь, содержащий все акционные паки со всеми периодами:

In [6]:
        action_pack={}
        for it in pcks:
            times=[]
            # 1) составляем временные промежутки для паков

            with open('trans.csv', 'rb') as csvfile:
                spamreader = csv.reader(csvfile, delimiter='|', quotechar='|')
                spamreader.next()
                for row in spamreader:
                    if row[5]==it:
                        times.append(int(row[2]))

            difference=172800+43200 # = 2.5 days in seconds
            # сравниваем даты по timestamp
            # если покупки наблюдались в окрестности 2х дней, то
            # будем считать, что в эти дни акция действовала
            times.sort()

            start_time=[]
            end_time=[]

            beg=times[0]

            start_time.append(beg)
            for item in times:
                if beg+difference>=item:
                    beg=item
                else:
                    start_time.append(item)
                    end_time.append(beg)
                    beg=item
            end_time.append(beg)

            # 2) проверяем временные промежутки; если все промежутки лежат в рамках 2.5 недель,
            # то считаем пак акционным и проходимся априори алгоритмом по выявленным промежуткам

            week_time=604800 # week time
            access=True
            for i in range(0,len(start_time)):
                if end_time[i]-start_time[i] > week_time*2.5: # предполагаем, что акции длятся не более 2х с половиной недель
                    access=False
            if access==True:
                action_pack[it]=[]
                action_pack[it].append(start_time)
                action_pack[it].append(end_time)
                
#action_pack


Априори для нашего файла. Соответсвенно возможны пересечения результатов (некоторые акции запускаются в одно и то же время).

In [7]:
    from datetime import datetime
    from datetime import date
    def write_res_to_file_name(file_name, mnsp, mncf, t_d):
        id_count=0
        now = datetime.now()
        minSupport=mnsp
        minConfidence=mncf
    
        count=0
        with open(file_name, 'w') as rs:
            for it,time_st_end in action_pack.items():
                start_time=time_st_end[0]
                end_time=time_st_end[1]
                for bound in range(0,len(start_time)):
                    ids={}
                    # 3) если пак акционный, то забиваем данные для составления ассоциативных правил

                    with open('trans.csv', 'rb') as csvfile:
                        spam = csv.reader(csvfile, delimiter='|', quotechar='|')
                        spam.next()
                        for row in spam:
                            if (int(row[2])>=start_time[bound] and int(row[2])<=end_time[bound]):
                                if ids.has_key(int(row[0]))==True:
                                    val=ids.get(int(row[0]))
                                    if row[5] not in val:
                                        val.append(row[5])
                                else:
                                    ids[int(row[0])]=[]# каждый id принимает в значение свои транзакции
                                    ids[int(row[0])].append(row[5])

                    with open('file_for_transactions.csv', 'w') as csf:
                        for key, value in ids.items():
                            st=""
                            for i in range(0,len(value)):
                                st=st+value[i]+','
                            csf.write(st+'\n')
                    #now1 = datetime.now()
                    #print "файлы: ","  ",(now1-now)

                    inFile = dataFromFile("file_for_transactions.csv")
                    items, rules = runApriori(inFile, minSupport, minConfidence)    
                    id_count, t_d=printResults(items, rules, rs,it, id_count, t_d, start_time[bound],end_time[bound],len(ids))
                    #print len(ids),"-",
                    


        # 4) время работы алгоритма
        now1 = datetime.now()
        print (now1-now)
        return t_d

Запускаем априори на наших данные. Подаем название файла, в который записываются результаты, minSupport, minConfidence.

In [8]:
tabl_dict={}
tabl_dic=write_res_to_file_name("res_l1_check_2.txt", 0.05, 0.6,tabl_dict)

0:04:29.343313


Создаем файл для tableau:

In [10]:
with open('good_thing_to_do.csv', 'w') as csvfile_1:
    fieldnames=["ID","Rule","Confidence","Lift","Support","Social_network","Date","Number_of_transactions_in_period","Consist_action_pack"]
    writer = csv.DictWriter(csvfile_1, fieldnames=fieldnames,delimiter='|')
    writer.writeheader()
    for key,value in tabl_dict.items():
        #print value[4],
        writer.writerow({fieldnames[0]: key, fieldnames[1]: value[0], fieldnames[2]: value[1], fieldnames[3]: value[2],fieldnames[4]: value[3],fieldnames[5]: value[4],fieldnames[6]: value[5],fieldnames[7]: value[6],fieldnames[8]: value[7]})

Результаты можно посомтреть по ссылке:
https://public.tableau.com/profile/publish/Rule_Inf_about_it/Dashboard3#!/publish-confirm

Основные моменты: чем больше транзакций, тем лучше (на наших данных, при кол-ве транзакций >100, в большинстве это правила внутри соц. сети - одноклассники), нужно исключить лифт = 1 (иначе данные независимы).

Также можно проследить в каких правилах учавствуют акционные паки. Если выделить только эти правила, то можно заметить, что во всех периодах наблюдается слишком мало транзакций. Такой результат можно интерпретировать либо как то, что в большинстве акции не влияют на результаты покупок (если, к примеру, акции должны приводить к покупкам данных паков пользователей с различных соц.сетей), либо, как то, что необходимо рассматривать правила по некоторым дополнительным признакам (как, к примеру, внутри одной соц. сети), т.к. иначе акционные паки не проходят порог минимального саппорта, который в нашем случае итак слишком маленький.

## Теперь хотим посомтреть ассоцитивные правила, включая социальные данные. Пока что будем включать только страну.

Обработка файла:
Csv библиотека не воспринимает "||", поэтому добавляем пробелы между каждыми "||".

In [58]:
count=1
with open('w_pinfo.csv', 'w') as csv:
    with open('wlitems_pinfo.csv', 'rb') as csvfile:
        for line in csvfile:
                temp=""
                if "||" in line:
                    for i in range(0,len(line)):
                        temp=temp+line[i]
                        if i!=len(line)-1 and line[i]=="|":
                            if line[i+1]=="|":
                                temp=temp+" "
                    csv.write(temp)
                else:
                    csv.write(line)

Создаем словарь айди-соц данные. Пока что тут лежит только страна. Ошибка выдается на нечисловой айдишник.

In [13]:
import sys, errno

now = datetime.now()
# 4 - country
id_soc_data={}
with open('w_pinfo.csv', 'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter='|', quotechar='|')
    spamreader.next()
    for row in spamreader:
        try:
            if id_soc_data.has_key(int(row[0]))==False:
                id_soc_data[int(row[0])]=[]
                id_soc_data[int(row[0])].append(row[5])
        except ValueError as e:
            print("Error has been found") #вылез странный айдишник 2.392...Е, пока что кидаю в еррор
            now1 = datetime.now()
            print(now1-now)
            now = datetime.now()
now1 = datetime.now()
print (now1-now)

Erros has been found
0:00:05.777879
0:00:01.784865


Априори на наших данных+страна.

In [ ]:
def printResults(items, rules, rs,it):
    """prints the generated itemsets sorted by support and the confidence rules sorted by confidence"""
    if len(rules)!=0:
        rs.write("SUP------------SUP------------SUP:\n")
        rs.write("For time boundaries on: "+it+"\n") #показывает периоды какого пака исследуются
        for item, support in sorted(items, key=lambda (item, support): support):
            rs.write("item: %s , %.3f \n" % (str(item), support))
            #print "item: %s , %.3f" % (str(item), support)
        rs.write("\nRULES------------RULES------------RULES:\n")
        for rule, confidence in sorted(rules, key=lambda (rule, confidence): confidence):
            pre, post = rule
            rs.write("Rule: %s ==> %s , confidence: %.3f, lift: %.3f\n" % (str(pre), str(post), confidence[0], confidence[1]))
        rs.write("\n-----------------------------------------\n")
        rs.write("\n")

In [22]:
from datetime import datetime
def write_res_to_file_name_with_additional_data(file_name, mnsp, mncf):
    now = datetime.now()
    minSupport=mnsp
    minConfidence=mncf
    
    count=0
    with open(file_name, 'w') as rs:
        for it in pcks:
            times=[]
            # 1) составляем временные промежутки для паков

            with open('trans.csv', 'rb') as csvfile:
                spamreader = csv.reader(csvfile, delimiter='|', quotechar='|')
                spamreader.next()
                for row in spamreader:
                    if row[5]==it:
                        times.append(int(row[2]))

            difference=172800+43200 # = 2.5 days in seconds
            # сравниваем даты по timestamp
            # если покупки наблюдались в окрестности 2х дней, то
            # будем считать, что в эти дни акция действовала
            times.sort()

            start_time=[]
            end_time=[]

            beg=times[0]

            start_time.append(beg)
            for item in times:
                if beg+difference>=item:
                    beg=item
                else:
                    start_time.append(item)
                    end_time.append(beg)
                    beg=item
            end_time.append(beg)

            # 2) проверяем временные промежутки; если все промежутки лежат в рамках 2.5 недель,
            # то считаем пак акционным и проходимся априори алгоритмом по выявленным промежуткам

            week_time=604800 # week time
            access=True
            for i in range(0,len(start_time)):
                if end_time[i]-start_time[i] > week_time*2.5: # предполагаем, что акции длятся не более 2х с половиной недель
                    access=False
                    count=count+1

            if access==True:
                for bound in range(0,len(start_time)):
                    ids={}
                    # 3) если пак акционный, то забиваем данные для составления ассоциативных правил

                    with open('trans.csv', 'rb') as csvfile:
                        spam = csv.reader(csvfile, delimiter='|', quotechar='|')
                        spam.next()
                        for row in spam:
                            if (int(row[2])>=start_time[bound] and int(row[2])<=end_time[bound]):
                                if ids.has_key(int(row[0]))==True:
                                    val=ids.get(int(row[0]))
                                    if row[5] not in val:
                                        val.append(row[5])
                                else:
                                    ids[int(row[0])]=[]# каждый id принимает в значение свои транзакции
                                    ids[int(row[0])].append(row[5])
                    
                    with open('file_for_transactions.csv', 'w') as csf:
                        for key, value in ids.items():
                            st=""
                            for i in range(0,len(value)):
                                st=st+value[i]+','
                            if id_soc_data.has_key(key):
                                if id_soc_data[key][0]!=" ":    
                                    st=st+id_soc_data[key][0]+","
                            csf.write(st+'\n')
                    #now1 = datetime.now()
                    #print "файлы: ","  ",(now1-now)

                    inFile = dataFromFile("file_for_transactions.csv")
                    items, rules = runApriori(inFile, minSupport, minConfidence)    
                    printResults(items, rules, rs, it)


    # 4) время работы алгоритма
    now1 = datetime.now()
    print (now1-now)

In [23]:
write_res_to_file_name_with_additional_data("res_ad_data_l1.txt", 0.2, 0.7)

0:04:28.286215


Когда рассматриваются все паки, результатом становятся достаточно очевидные вещи, как для US => fb, RU => vk/ok. Рассмотрим внутри одной соцсети - fb. 
Всего fb паков:

In [30]:
#social_netork='fb'
social_network="fb"
pcks=[]
with open('trans.csv', 'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter='|', quotechar='|')
    pcks=[]
    spamreader.next()
    for row in spamreader:
        if row[1]==social_network:
            if (row[5] not in pcks):
                pcks.append(row[5])
    print len(pcks)

54


Ассоц. правила только по fb:

In [31]:
from datetime import datetime

def write_res_to_file_name_with_additional_data_concrete_soc(file_name, mnsp, mncf):
    now = datetime.now()
    minSupport=mnsp
    minConfidence=mncf
    
    count=0
    with open(file_name, 'w') as rs:
        for it in pcks:
            times=[]
            # 1) составляем временные промежутки для паков

            with open('trans.csv', 'rb') as csvfile:
                spamreader = csv.reader(csvfile, delimiter='|', quotechar='|')
                spamreader.next()
                for row in spamreader:
                    if row[5]==it:
                        times.append(int(row[2]))

            difference=172800+43200 # = 2.5 days in seconds
            # сравниваем даты по timestamp
            # если покупки наблюдались в окрестности 2х дней, то
            # будем считать, что в эти дни акция действовала
            times.sort()

            start_time=[]
            end_time=[]

            beg=times[0]

            start_time.append(beg)
            for item in times:
                if beg+difference>=item:
                    beg=item
                else:
                    start_time.append(item)
                    end_time.append(beg)
                    beg=item
            end_time.append(beg)

            # 2) проверяем временные промежутки; если все промежутки лежат в рамках 2.5 недель,
            # то считаем пак акционным и проходимся априори алгоритмом по выявленным промежуткам

            week_time=604800 # week time
            access=True
            for i in range(0,len(start_time)):
                if end_time[i]-start_time[i] > week_time*2.5: # предполагаем, что акции длятся не более 2х с половиной недель
                    access=False
                    count=count+1

            if access==True:
                for bound in range(0,len(start_time)):
                    ids={}
                    # 3) если пак акционный, то забиваем данные для составления ассоциативных правил

                    with open('trans.csv', 'rb') as csvfile:
                        spam = csv.reader(csvfile, delimiter='|', quotechar='|')
                        spam.next()
                        for row in spam:
                            if row[1]==social_network:
                                if (int(row[2])>=start_time[bound] and int(row[2])<=end_time[bound]):
                                    if ids.has_key(int(row[0]))==True:
                                        val=ids.get(int(row[0]))
                                        if row[5] not in val:
                                            val.append(row[5])
                                    else:
                                        ids[int(row[0])]=[]# каждый id принимает в значение свои транзакции
                                        ids[int(row[0])].append(row[5])
                    
                    with open('file_for_transactions.csv', 'w') as csf:
                        for key, value in ids.items():
                            st=""
                            for i in range(0,len(value)):
                                st=st+value[i]+','
                            if id_soc_data.has_key(key):
                                if id_soc_data[key][0]!=" ":    
                                    st=st+id_soc_data[key][0]+","
                            csf.write(st+'\n')
                    #now1 = datetime.now()
                    #print "файлы: ","  ",(now1-now)

                    inFile = dataFromFile("file_for_transactions.csv")
                    items, rules = runApriori(inFile, minSupport, minConfidence)    
                    printResults(items, rules, rs, it)


    # 4) время работы алгоритма
    now1 = datetime.now()
    print (now1-now)

In [32]:
write_res_to_file_name_with_additional_data_concrete_soc("res_l1_fb.txt", 0.2, 0.7)

0:01:06.599595
